***Cell 1: Setup - Imports and API Configuration***

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output
import markdown
import re
import uuid

# --- API Configuration ---
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("Google API key not found in .env file.")
else:
    genai.configure(api_key=google_api_key)
    print("✅ Google Gemini client configured successfully.")

✅ Google Gemini client configured successfully.


***Cell 2: The Control Panel (Interactive Widgets & Layout)***

In [5]:
# --- Section 1: Model & Creativity Parameters ---
title_model = widgets.HTML("<h3>1. Model & Creativity Settings</h3>")

model_selector = widgets.Dropdown(
    options=['gemini-2.5-flash', 'gemini-1.5-flash-latest', 'gemini-1.5-pro'],
    value='gemini-1.5-flash-latest',
    description='Model:',
    layout={'width': 'auto'}
)
temperature_slider = widgets.FloatSlider(
    value=0.7, min=0, max=2.0, step=0.1, description='Temperature:', readout_format='.1f', layout={'width': 'auto'}
)
top_p_slider = widgets.FloatSlider(
    value=0.95, min=0, max=1.0, step=0.01, description='Top-P:', readout_format='.2f', layout={'width': 'auto'}
)
top_k_slider = widgets.IntSlider(
    value=40, min=1, max=100, step=1, description='Top-K:', layout={'width': 'auto'}
)
param_tip = widgets.HTML("<p><i><b>Pro Tip:</b> For best results, adjust either <b>Temperature</b> OR <b>Top-P/Top-K</b>, not all at once.</i></p>")

model_creativity_box = widgets.HBox([model_selector, temperature_slider, top_p_slider, top_k_slider])

# --- Section 2: Output Control ---
title_output = widgets.HTML("<h3>2. Output Control</h3>")
max_tokens_slider = widgets.IntSlider(
    value=2048, min=256, max=8192, step=256, description='Max Tokens:'
)

# --- Section 3: Prompt Definition ---
title_prompts = widgets.HTML("<h3>3. Prompt Definition</h3>")
system_prompt_input = widgets.Textarea(
    value='You are a helpful and creative AI assistant that writes Python code.',
    description='System Prompt:', layout={'height': '100px', 'width': '99%'}
)
user_prompt_input = widgets.Textarea(
    value='Write a Python function that takes a list of numbers and returns a new list with only the even numbers.',
    description='User Prompt:', layout={'height': '200px', 'width': '99%'}
)

# --- Section 4: Execution & Output ---
title_execution = widgets.HTML("<h3>4. Execution</h3>")
run_button = widgets.Button(description='Generate Response', button_style='success', icon='rocket', layout={'width': '200px'})
output_area = widgets.Output()

# --- Final Layout Assembly ---
control_panel = widgets.VBox([
    title_model,
    model_creativity_box,
    param_tip,
    title_output,
    max_tokens_slider,
    title_prompts,
    system_prompt_input,
    user_prompt_input,
    title_execution,
    run_button,
    output_area
])

display(control_panel)

***Cell 3: The Execution Logic (The "Engine") with Advanced Rendering***

In [6]:
import uuid
from pygments.formatters import HtmlFormatter
import markdown
import re

def generate_response_with_streaming(b):
    """
    Main execution function, triggers on button click. This final version ensures
    proper markdown rendering for all parts.
    """
    with output_area:
        clear_output(wait=True)
        print("🚀 Sending request to Gemini...")

        generation_config = {
            "temperature": temperature_slider.value, "top_p": top_p_slider.value,
            "top_k": top_k_slider.value, "max_output_tokens": max_tokens_slider.value,
        }
        
        try:
            model = genai.GenerativeModel(
                model_selector.value,
                system_instruction=system_prompt_input.value
            )
            # We will use non-streaming for the final version to simplify rendering logic
            response = model.generate_content(
                user_prompt_input.value,
                generation_config=generation_config
            )
            
            clear_output(wait=True)
            print("--- ✅ Gemini's Response ---")

            full_response_text = response.text
            
            if not full_response_text.strip():
                display(widgets.HTML("<i>The model returned no content.</i>"))
                return

            # --- Final Rendering Logic ---
            
            # 1. Generate CSS for syntax highlighting
            css = HtmlFormatter(style='default').get_style_defs('.codehilite')

            # 2. Split the text by code blocks
            parts = re.split(r'(```(?:\w+\n)?[\s\S]*?```)', full_response_text)
            
            for part in parts:
                if not part.strip():
                    continue

                if part.startswith('```'):
                    # --- RENDER A CODE BLOCK ---
                    code_content = re.sub(r'```(?:\w+\n)?', '', part).rstrip('`')
                    
                    # Convert just the code to highlighted HTML
                    highlighted_code_html = markdown.markdown(f"```{code_content}", extensions=['fenced_code', 'codehilite'])
                    
                    text_area_id = f"code-area-{uuid.uuid4()}"
                    
                    # Create the button with self-contained, robust JS
                    button_html = f"""
                    <textarea id='{text_area_id}' style='position: absolute; left: -9999px;'>{code_content}</textarea>
                    <button onclick="
                        var text = document.getElementById('{text_area_id}').value;
                        navigator.clipboard.writeText(text).then(() => {{
                            this.innerHTML = 'Copied!';
                            setTimeout(() => {{ this.innerHTML = 'Copy Code'; }}, 2000);
                        }});
                    " style='position: absolute; top: 8px; right: 8px; border: 1px solid #ccc; border-radius: 4px; padding: 2px 8px; cursor: pointer; font-size: 12px;'>Copy Code</button>
                    """
                    
                    # Assemble the final HTML for this block inside a container
                    final_html = f"""
                    <style>{css}</style>
                    <div style='position: relative; border: 1px solid #e0e0e0; border-radius: 5px; background-color: #f9f9f9; padding-top: 5px; margin-bottom:10px;'>
                        {button_html}
                        {highlighted_code_html}
                    </div>
                    """
                    display(widgets.HTML(value=final_html))

                else:
                    # --- RENDER A TEXT BLOCK ---
                    text_html = markdown.markdown(part)
                    display(widgets.HTML(value=f"<div style='margin-bottom:10px;'>{text_html}</div>"))

        except Exception as e:
            clear_output(wait=True)
            display(widgets.HTML(value=f"<div style='border: 1px solid red; padding: 15px; color: red;'><b>An error occurred:</b><br>{e}</div>"))

***Cell 4: Connecting the Button to the Logic***

In [7]:
# This line "connects" the button to our function.
# When the button is clicked, the 'generate_response' function will be executed.
run_button.on_click(generate_response_with_streaming)